In [41]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,699 kB]

In [42]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-12-13 20:05:08--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.81MB/s    in 0.2s    

2020-12-13 20:05:08 (4.81 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [57]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [58]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [59]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [60]:
# Create the vine_table. DataFrame

vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase']).drop_duplicates()
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|                Y|
| R8VY6DVIII3YM|          5|            0|          0|   N|                Y|
| R2J67NHCAOCO5|          5|            0|          0|   N|                Y|
| R5J9HUTA1U143|          4|            0|          0|   N|                Y|
|R2I858T97JS6CA|          3|            0|          0|   N|                Y|
|R2QMKKO1LDGEDX|          5|            0|          0|   N|                Y|
|R1PV81CMC2GCQY|          5|            0|          0|   N|                Y|
|R3HE5JN1SSNQPU|          5|            0|          0|   N|                Y|
|R1GDPQGWQ2TQIU|          5|            0|          0|   N|     

In [61]:
greater_than_20 = vine_df.filter(vine_df.total_votes >=20)
greater_than_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2XLHR2NF0B6VE|          1|           17|         37|   N|                N|
|R3PCZZC2WI7DIS|          5|           31|         37|   N|                N|
|R2TF47XY2GCXMU|          3|            1|         51|   N|                N|
|R3KPTBOWWL80G5|          1|           10|         39|   N|                N|
|R3BID0BEH0LU0A|          3|           41|         51|   N|                N|
|R3IDAQS8KD7IWW|          1|            3|        130|   N|                N|
| RKCTZMF9CBL8A|          5|           19|         26|   N|                Y|
|R2XDFJV657764G|          1|            3|         39|   N|                N|
|R3SXT1S3V59CI0|          1|            1|         26|   N|                N|
|R208RABD241FMX|          5|           21|         26|   N|     

In [62]:
helpful_df = greater_than_20.filter(greater_than_20.helpful_votes/greater_than_20.total_votes >= 0.5)
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PCZZC2WI7DIS|          5|           31|         37|   N|                N|
|R3BID0BEH0LU0A|          3|           41|         51|   N|                N|
| RKCTZMF9CBL8A|          5|           19|         26|   N|                Y|
|R208RABD241FMX|          5|           21|         26|   N|                N|
| RTLR1RG87TL9A|          3|           56|         65|   N|                Y|
|R114G1R3U71DCO|          5|           28|         38|   N|                N|
| RUE17Q5WXLZ3M|          3|           43|         49|   N|                N|
|R376FG23YUFWH5|          4|           71|         71|   N|                Y|
|R2RT9LFGKFLX9P|          5|           79|         99|   N|                Y|
|R14VW0B3JSTA38|          2|           22|         27|   N|     

In [63]:
from pyspark.sql import functions as func

helpful_df = helpful_df.withColumn('Review_type', func.when(func.col('vine') == 'Y', "Vine Reviewed").otherwise("Non Vine"))
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Review_type|
+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|R3PCZZC2WI7DIS|          5|           31|         37|   N|                N|   Non Vine|
|R3BID0BEH0LU0A|          3|           41|         51|   N|                N|   Non Vine|
| RKCTZMF9CBL8A|          5|           19|         26|   N|                Y|   Non Vine|
|R208RABD241FMX|          5|           21|         26|   N|                N|   Non Vine|
| RTLR1RG87TL9A|          3|           56|         65|   N|                Y|   Non Vine|
|R114G1R3U71DCO|          5|           28|         38|   N|                N|   Non Vine|
| RUE17Q5WXLZ3M|          3|           43|         49|   N|                N|   Non Vine|
|R376FG23YUFWH5|          4|           71|         71|   N|                Y|   Non Vine|
|R2RT9LFGK

In [64]:
helpful_df.groupby('Review_type').count().show()

+-------------+-----+
|  Review_type|count|
+-------------+-----+
|     Non Vine|41535|
|Vine Reviewed|   98|
+-------------+-----+



In [65]:
vine_reviews = helpful_df.filter(helpful_df.vine=='Y')
vine_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  Review_type|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
| REQZP78LTC0EY|          4|           21|         23|   Y|                N|Vine Reviewed|
|R1FE1OLQ4T3T1Q|          4|          179|        182|   Y|                N|Vine Reviewed|
|  RV8FVXVWTEAX|          4|           97|        107|   Y|                N|Vine Reviewed|
|R1X29HLKI4XO6C|          4|          331|        357|   Y|                N|Vine Reviewed|
|R147GE7N7DBNSK|          5|           55|         60|   Y|                N|Vine Reviewed|
|R15ARADYJE29FK|          3|           33|         36|   Y|                N|Vine Reviewed|
|R2INH817LTJFGI|          5|           18|         20|   Y|                N|Vine Reviewed|
|R3CMWDEYCJM2O9|          1|           24|         31|   Y|                N|Vin

In [66]:
non_vine_reviews = helpful_df.filter(helpful_df.vine=='N')
non_vine_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Review_type|
+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|R3PCZZC2WI7DIS|          5|           31|         37|   N|                N|   Non Vine|
|R3BID0BEH0LU0A|          3|           41|         51|   N|                N|   Non Vine|
| RKCTZMF9CBL8A|          5|           19|         26|   N|                Y|   Non Vine|
|R208RABD241FMX|          5|           21|         26|   N|                N|   Non Vine|
| RTLR1RG87TL9A|          3|           56|         65|   N|                Y|   Non Vine|
|R114G1R3U71DCO|          5|           28|         38|   N|                N|   Non Vine|
| RUE17Q5WXLZ3M|          3|           43|         49|   N|                N|   Non Vine|
|R376FG23YUFWH5|          4|           71|         71|   N|                Y|   Non Vine|
|R2RT9LFGK

In [67]:
# Vine Statistics
vine_count = vine_reviews.count()
vine_5_stars = vine_reviews.filter(vine_reviews.star_rating ==5).count()
vine_5_star_pct = round(vine_5_stars/vine_count*100,1)

In [68]:
# Non Vine Statistics
non_vine_count = non_vine_reviews.count()
non_vine_5_stars = non_vine_reviews.filter(non_vine_reviews.star_rating ==5).count()
non_vine_5_star_pct = round(non_vine_5_stars/non_vine_count*100,1)

In [69]:
vine_summary = spark.createDataFrame([
                                   ("Vine Reviewed", vine_count, vine_5_stars, vine_5_star_pct),
                                   ("Non-Vine", non_vine_count, non_vine_5_stars, non_vine_5_star_pct),], 
                                 ["Review Type", "Total Reviews", "Total 5-star Rating Reviews", "% of 5-star Reviews"])


In [70]:
vine_summary.show()

+-------------+-------------+---------------------------+-------------------+
|  Review Type|Total Reviews|Total 5-star Rating Reviews|% of 5-star Reviews|
+-------------+-------------+---------------------------+-------------------+
|Vine Reviewed|           98|                         49|               50.0|
|     Non-Vine|        41535|                      16190|               39.0|
+-------------+-------------+---------------------------+-------------------+

